In [ ]:
# Define helper functions

# CNF formula for exactly one of the variables in list A to be true
def exactly_one(A):
    temp = ""
    temp += atleast_one(A)
    temp += atmost_one(A)
    return temp

# CNF formula for at least one of the variables in list A to be true
def atleast_one(A):
    temp = ""
    for x in A:
        temp += " " + str(x)
    temp += " 0\n"
    return temp

# CNF formula for at most one of the variables in list A to be true
def atmost_one(A):
    temp = ""
    for i in range(len(A)):
        for j in range(i+1, len(A)):
            temp += " -" + str(A[i]) + " -" + str(A[j]) + " 0\n"
    return temp

# Function to map position (r,c) 0 <= r,c < N, in an NxN grid to the integer
# position when the grid is stored linearly by rows.
def varmap(r, c, N):
    return r * N + c + 1

# Set board size N
N = 4  # Default to 4-Queens

# Start Solver
spots = N * N

# Store all constraints in a single string
temp = ""

# Exactly 1 queen per row
for row in range(N):
    A = []
    for column in range(N):
        position = varmap(row, column, N)
        A.append(position)
    temp += exactly_one(A)

# Exactly 1 queen per column
for column in range(N):
    A = []
    for row in range(N):
        position = varmap(row, column, N)
        A.append(position)
    temp += exactly_one(A)

# At most 1 queen per negative diagonal from left
for row in range(N-1, -1, -1):
    A = []
    for x in range(0, N-row):
        A.append(varmap(row+x, x, N))
    temp += atmost_one(A)

# At most 1 queen per negative diagonal from top
for column in range(1, N):
    A = []
    for x in range(0, N-column):
        A.append(varmap(x, column+x, N))
    temp += atmost_one(A)

# At most 1 queen per positive diagonal from right
for row in range(N-1, -1, -1):
    A = []
    for x in range(0, N-row):
        A.append(varmap(row+x, N-1-x, N))
    temp += atmost_one(A)

# At most 1 queen per positive diagonal from top
for column in range(N-2, -1, -1):
    A = []
    for x in range(0, column+1):
        A.append(varmap(x, column-x, N))
    temp += atmost_one(A)

# Prepare CNF output
cnf_output = "p cnf " + str(N*N) + " " + str(temp.count('\n')) +"\n"+ temp

# Display the CNF output
print(cnf_output)

p cnf 16 84
 1 2 3 4 0
 -1 -2 0
 -1 -3 0
 -1 -4 0
 -2 -3 0
 -2 -4 0
 -3 -4 0
 5 6 7 8 0
 -5 -6 0
 -5 -7 0
 -5 -8 0
 -6 -7 0
 -6 -8 0
 -7 -8 0
 9 10 11 12 0
 -9 -10 0
 -9 -11 0
 -9 -12 0
 -10 -11 0
 -10 -12 0
 -11 -12 0
 13 14 15 16 0
 -13 -14 0
 -13 -15 0
 -13 -16 0
 -14 -15 0
 -14 -16 0
 -15 -16 0
 1 5 9 13 0
 -1 -5 0
 -1 -9 0
 -1 -13 0
 -5 -9 0
 -5 -13 0
 -9 -13 0
 2 6 10 14 0
 -2 -6 0
 -2 -10 0
 -2 -14 0
 -6 -10 0
 -6 -14 0
 -10 -14 0
 3 7 11 15 0
 -3 -7 0
 -3 -11 0
 -3 -15 0
 -7 -11 0
 -7 -15 0
 -11 -15 0
 4 8 12 16 0
 -4 -8 0
 -4 -12 0
 -4 -16 0
 -8 -12 0
 -8 -16 0
 -12 -16 0
 -9 -14 0
 -5 -10 0
 -5 -15 0
 -10 -15 0
 -1 -6 0
 -1 -11 0
 -1 -16 0
 -6 -11 0
 -6 -16 0
 -11 -16 0
 -2 -7 0
 -2 -12 0
 -7 -12 0
 -3 -8 0
 -12 -15 0
 -8 -11 0
 -8 -14 0
 -11 -14 0
 -4 -7 0
 -4 -10 0
 -4 -13 0
 -7 -10 0
 -7 -13 0
 -10 -13 0
 -3 -6 0
 -3 -9 0
 -6 -9 0
 -2 -5 0



In [1]:
from pysat.solvers import Glucose3

# Create a SAT Solver instance (Glucose3)
with Glucose3() as g:
    # Add clauses to the solver
    g.add_clause([1, -5, 4]) 
    g.add_clause([-1, 5, 3, 4])   
    g.add_clause([-3, -4])

    # Solve the SAT problem
    if g.solve():
        print("SATISFIABLE")
        print("Model:", g.get_model())  # Get the model (satisfying assignment)
    else:
        print("UNSATISFIABLE")

SATISFIABLE
Model: [-1, -2, -3, -4, -5]


AMO - Binomial encoding

- Variables: the same $x_0, \ldots, x_n$
$\rightarrow$ no new variables
- Clauses: for $0 \leq \mathrm{i}<\mathrm{j}<\mathrm{n}, \neg \mathrm{x}_{\mathrm{j}} \vee \neg \mathrm{x}_{\mathrm{j}}\left(\mathrm{x}_{\mathrm{i}} \rightarrow \neg \mathrm{x}_{\mathrm{j}}\right)$
- Requires: $O\left(n^2\right)$ clauses

ALO: $x_0 \vee x_1 \vee \ldots \vee x_n$

In [10]:
# binomial

def generate_variables(n):
    return [[i * n + j + 1 for j in range(n)] for i in range(n)]


def at_most_one(clauses, variables):
    for i in range(0, len(variables)):
        for j in range(i + 1, len(variables)):
            clauses.append([-variables[i], -variables[j]])
    return clauses


def exactly_one(clauses, variables):
    clauses.append(variables)
    at_most_one(clauses, variables)


def generate_clauses(n, variables):
    clauses = []

    # Exactly one queen in each row
    for i in range(n):
        exactly_one(clauses, variables[i])

    # Exactly one queen in each column
    for j in range(n):
        exactly_one(clauses, [variables[i][j] for i in range(n)])


    # At most one queen in each diagonal
    for i in range(n):
        for j in range(n):
            for k in range(1, n):
                if i + k < n and j + k < n:
                    at_most_one(clauses, [variables[i][j], variables[i + k][j + k]])
                if i + k < n and j - k >= 0:
                    at_most_one(clauses, [variables[i][j], variables[i + k][j - k]])

    return clauses


def solve_n_queens(n):
    variables = generate_variables(n)
    clauses = generate_clauses(n, variables)
    print(clauses)

    solver = Glucose3()
    for clause in clauses:
        solver.add_clause(clause)

    if solver.solve():
        model = solver.get_model()
        return [[int(model[i * n + j] > 0) for j in range(n)] for i in range(n)]
    else:
        return None


def print_solution(solution):
    if solution is None:
        print("No solution found.")
    else:
        print(solution)
        for row in solution:
            print(" ".join("Q" if cell else "." for cell in row))


n = 8
# print(generate_clauses(n, generate_variables(n)))
solution = solve_n_queens(n)
print_solution(solution)

[[1, 2, 3, 4, 5, 6, 7, 8], [-1, -2], [-1, -3], [-1, -4], [-1, -5], [-1, -6], [-1, -7], [-1, -8], [-2, -3], [-2, -4], [-2, -5], [-2, -6], [-2, -7], [-2, -8], [-3, -4], [-3, -5], [-3, -6], [-3, -7], [-3, -8], [-4, -5], [-4, -6], [-4, -7], [-4, -8], [-5, -6], [-5, -7], [-5, -8], [-6, -7], [-6, -8], [-7, -8], [9, 10, 11, 12, 13, 14, 15, 16], [-9, -10], [-9, -11], [-9, -12], [-9, -13], [-9, -14], [-9, -15], [-9, -16], [-10, -11], [-10, -12], [-10, -13], [-10, -14], [-10, -15], [-10, -16], [-11, -12], [-11, -13], [-11, -14], [-11, -15], [-11, -16], [-12, -13], [-12, -14], [-12, -15], [-12, -16], [-13, -14], [-13, -15], [-13, -16], [-14, -15], [-14, -16], [-15, -16], [17, 18, 19, 20, 21, 22, 23, 24], [-17, -18], [-17, -19], [-17, -20], [-17, -21], [-17, -22], [-17, -23], [-17, -24], [-18, -19], [-18, -20], [-18, -21], [-18, -22], [-18, -23], [-18, -24], [-19, -20], [-19, -21], [-19, -22], [-19, -23], [-19, -24], [-20, -21], [-20, -22], [-20, -23], [-20, -24], [-21, -22], [-21, -23], [-21, -24

In [ ]:
# binomial

def solve_nqueens(n):
    with Glucose3() as g:
        # Create variables: vars[i][j] represents queen at position (i,j)
        vars = [[(i * n + j + 1) for j in range(n)] for i in range(n)]

        # Constraint 1: Exactly one queen per row
        for i in range(n):
            g.add_clause(vars[i])  # At least one queen in row
            # At most one queen (pairwise negative clauses)
            for j in range(n):
                for k in range(j + 1, n):
                    g.add_clause([-vars[i][j], -vars[i][k]])

        # Constraint 2: Exactly one queen per column
        for j in range(n):
            col = [vars[i][j] for i in range(n)]
            g.add_clause(col)  # At least one queen in column
            # At most one queen (pairwise negative clauses)
            for i in range(n):
                for k in range(i + 1, n):
                    g.add_clause([-vars[i][j], -vars[k][j]])

        # Constraint 3: At most one queen per diagonal
        for i1 in range(n):
            for j1 in range(n):
                for i2 in range(n):
                    for j2 in range(n):
                        if (i1 != i2 and j1 != j2) and (abs(i1 - i2) == abs(j1 - j2)):
                            g.add_clause([-vars[i1][j1], -vars[i2][j2]])

        # Solve and output result
        if g.solve():
            model = g.get_model()
            board = [['.' for _ in range(n)] for _ in range(n)]
            for i in range(n):
                for j in range(n):
                    if vars[i][j] in model:
                        board[i][j] = 'Q'
            for row in board:
                print(' '.join(row))
        else:
            print("UNSATISFIABLE")

# Solve for n=4
solve_nqueens(8)

. . . Q . . . .
. Q . . . . . .
. . . . . . Q .
. . Q . . . . .
. . . . . Q . .
. . . . . . . Q
. . . . Q . . .
Q . . . . . . .


Binary encoding 

- Idea:
- Variables: $X_i$ and new $k$ variables $Y_1, Y_2, \ldots, Y_k(k=$ $\left[\log _2 n\right]$)$
    - $X_i$ corresponding a binary string
- Clauses: $X_i \leftrightarrow \varphi(i, 1) \wedge \varphi(i, 2) \wedge \ldots \wedge \varphi(i, k)$ where
    - $\varphi(i, j)$ denotes for $Y_j$ if $j$-th digit in binary string is 1
- $\varphi(i, j)$ denotes for $\neg Y_j$ otherwise
    - Requires: $O\left(\log _2{ }^n\right)$ new variables, $O\left(n^* \log _2{ }^n\right)$ clauses


- For $n=4\left(X_1, X_2, X_3, X_4\right)$
    - $\mathrm{X}_1$ corresponding " 00 " $\quad \mathrm{X}_1 \leftrightarrow \neg \mathrm{Y}_1 \wedge \neg \mathrm{Y}_2$
    - $\mathrm{X}_2$ corresponding " 01 " $\quad \mathrm{X}_2 \leftrightarrow \mathrm{Y}_1 \wedge \neg \mathrm{Y}_2$
    - $\mathrm{X}_3$ corresponding " 10 " $\quad \mathrm{X}_3 \leftrightarrow \neg \mathrm{Y}_1 \wedge \mathrm{Y}_2$
    - $\mathrm{X}_4$ corresponding " 11 " $\mathrm{X}_4 \leftrightarrow \mathrm{Y}_1 \wedge \mathrm{Y}_2$
- AMO Encoding:

$$
\left(X_1 \rightarrow \neg Y_1 \wedge \neg Y_2\right) \wedge\left(X_2 \rightarrow Y_1 \wedge \neg Y_2\right) \wedge\left(X_3 \rightarrow \neg Y_1 \wedge Y_2\right) \wedge\left(X_4 \rightarrow Y_1 \wedge Y_2\right)
$$

- ALO Encoding:
    - (i): $\left(\neg Y_1 \wedge \neg Y_2 \rightarrow X_1\right) \wedge\left(Y_1 \wedge \neg Y_2 \rightarrow X_2\right) \wedge\left(\neg Y_1 \wedge Y_2 \rightarrow X_3\right) \wedge\left(Y_1 \wedge Y_2 \rightarrow X_4\right)$
    - (ii): $\left(X_1 \vee X_2 \vee X_3 \vee X_4\right)$


In [8]:
#binary


def generate_variables(n):
    return [[i * n + j + 1 for j in range(n)] for i in range(n)]


def binary_encoding(clauses, target, new_variables):
    for i in range(len(new_variables)):
        clauses.append([-target, new_variables[i]])


def generate_binary_combinations(n):
    binary_combinations = []
    for i in range(1 << n):
        binary_combinations.append(format(i, '0' + str(n) + 'b'))
    return binary_combinations

def generate_new_variables(end, length):
    return [i for i in range(end + 1, end + math.ceil(math.log(length, 2)) + 1)]


def at_most_one(clauses, new_variables, variables):
    temp_new_variables = generate_new_variables(n ** 2 + len(new_variables), len(variables))
    new_variables += temp_new_variables
    binary_combinations = generate_binary_combinations(len(temp_new_variables))

    for i in range(len(variables)):
        combination = binary_combinations[i]
        temp_clause = []
        for j in range(len(combination) - 1, -1, -1):
            index = len(combination) - j - 1
            temp_clause.append({True: temp_new_variables[index], False: -temp_new_variables[index]} [int(combination[j]) == 1])

        binary_encoding(clauses, variables[i], temp_clause)


def exactly_one(clauses, new_variables, variables):
    clauses.append(variables)
    at_most_one(clauses, new_variables, variables)


def generate_clauses(n, variables):
    clauses = []
    new_variables = []

    # Exactly one queen in each row
    for i in range(n):
        exactly_one(clauses, new_variables, variables[i])

    # Exactly one queen in each column
    for j in range(n):
        exactly_one(clauses, new_variables, [variables[i][j] for i in range(n)])


    # At most one queen in each diagonal
    for i in range(1, n):
        diagonal = []
        row = i
        col = 0
        while row >= 0 and col < n:
            diagonal.append(variables[row][col])
            row -= 1
            col += 1
        at_most_one(clauses, new_variables, diagonal)

    for j in range(1, n - 1):
        diagonal = []
        row = n - 1
        col = j
        while row >= 0 and col < n:
            diagonal.append(variables[row][col])
            row -= 1
            col += 1
        at_most_one(clauses, new_variables, diagonal)

    for i in range(n - 1):
        diagonal = []
        row = i
        col = 0
        while row < n and col < n:
            diagonal.append(variables[row][col])
            row += 1
            col += 1
        at_most_one(clauses, new_variables, diagonal)

    for j in range(1, n - 1):
        diagonal = []
        row = 0
        col = j
        while row < n and col < n:
            diagonal.append(variables[row][col])
            row += 1
            col += 1
        at_most_one(clauses, new_variables, diagonal)

    return clauses


def solve_n_queens(n):
    variables = generate_variables(n)
    clauses = generate_clauses(n, variables)

    solver = Glucose3()
    for clause in clauses:
        solver.add_clause(clause)

    if solver.solve():
        model = solver.get_model()
        return [[int(model[i * n + j] > 0) for j in range(n)] for i in range(n)]
    else:
        return None


def print_solution(solution):
    if solution is None:
        print("No solution found.")
    else:
        for row in solution:
            print(" ".join("Q" if cell else "." for cell in row))


n = 512
solution = solve_n_queens(n)
print_solution(solution)

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

Commander encoding

- Idea:
- Partitioning set of variables $X_1, X_2, \ldots, X_n$ into some groups of some fixed size and encoding AMO over each group
    - $X_1, X_2, \ldots, X_k \in G_1$
    - $X_{k+1}, X_{k+2}, \ldots, X_{2 k} \in G_2 ; \ldots$
    - $X_j, \ldots, X_n \in G_m$
- Introducing a commander variable for each group: $c_1$, $c_2, \ldots, c_m$
    - AMO, ALO Clauses for $c_1, c_2, \ldots, c_m$
    (Binomial encoding)
- If a commander variable $c_i$ is true then exactly one variable true in $\mathrm{G}_{\mathrm{i}}$
(AMO and ALO of $\mathrm{G}_{\mathrm{i}}$ )
    - If a commander variable $c_i$ is false then all variables false in $\mathrm{G}_{\mathrm{i}}$



- For $n=6\left(X_1 X_2, \ldots, X_6\right)$
    - $X_1, X_2, X_3 \in G_1$
    - $X_4, X_5, X_6 \in G_2$
    - $\left(c_1 \vee c_2\right) \wedge\left(c_1 \rightarrow \neg c_2\right) \wedge$
    - $c_1 \rightarrow\left(X_1 \vee X_2 \vee X_3\right) \wedge\left(X_1 \rightarrow \neg X_2\right) \wedge\left(X_1 \rightarrow \neg X_3\right) \wedge\left(X_2 \rightarrow \neg X_3\right) \wedge$
    - $c_2 \rightarrow\left(X_4 \vee X_5 \vee X_6\right) \wedge\left(X_4 \rightarrow \neg X_5\right) \wedge\left(X_4 \rightarrow \neg X_5\right) \wedge\left(X_5 \rightarrow \neg X_6\right) \wedge$
    - $\left(\neg c_1 \rightarrow \neg X_1\right) \wedge\left(\neg c_1 \rightarrow \neg X_2\right) \wedge\left(\neg c_1 \rightarrow \neg X_3\right) \wedge$
    - $\left(\neg c_2 \rightarrow \neg X_4\right) \wedge\left(\neg c_2 \rightarrow \neg X_5\right) \wedge\left(\neg c_2 \rightarrow \neg X_6\right)$
- For $n=7\left(X_1 X_2, \ldots, X_7\right)$?
    - $X_1, X_2, X_3, X_4 \in G_1$
    - $X_5, X_6, X_7 \in G_2$

In [ ]:

#conmmander

new_variables_count = 0
start_time = time.time()
def generate_variables(n):
    return [[i * n + j + 1 for j in range(n)] for i in range(n)]


def generate_new_variables(start, length):
    return [i for i in range(start, start + length)]

def generate_binary_combinations(n):
    binary_combinations = []
    for i in range(1 << n):
        binary_combinations.append(format(i, '0' + str(n) + 'b'))
    return binary_combinations


def binary_encoding(clauses, target, new_variables):
    for i in range(len(new_variables)):
        clauses.append([-target, new_variables[i]])

def generate_new_variables(end, length):
    return [i for i in range(end + 1, end + math.ceil(math.log(length, 2)) + 1)]

def binary_AMO(clauses, variables):
    global new_variables_count
    temp_new_variables = generate_new_variables(n ** 2 + new_variables_count, len(variables))
    new_variables_count += len(temp_new_variables)
    binary_combinations = generate_binary_combinations(len(temp_new_variables))

    for i in range(len(variables)):
        combination = binary_combinations[i]
        temp_clause = []
        for j in range(len(combination) - 1, -1, -1):
            index = len(combination) - j - 1
            temp_clause.append({True: temp_new_variables[index], False: -temp_new_variables[index]} [int(combination[j]) == 1])

        binary_encoding(clauses, variables[i], temp_clause)

def grouping_variables(variables, per_group):
    groups = []
    temp_groups = []
    for i in range(len(variables)):
        temp_groups.append(variables[i])
        if((i + 1) % per_group == 0 or i == len(variables) - 1):
            groups.append(temp_groups)
            temp_groups = []
    return groups

def generate_commanders(start, length):
    return [i for i in range(start, start + length)]


def binomial_AMO(clauses, variables):
    for i in range(0, len(variables)):
        for j in range(i + 1, len(variables)):
            clauses.append([-variables[i], -variables[j]])
    return clauses


def binomial_EO(clauses, variables):
    clauses.append(variables)
    binomial_AMO(clauses, variables)



def at_most_one(clauses, variables):
        global new_variables_count
        groups = grouping_variables(variables, n/math.floor(math.sqrt(n)))
        commanders = generate_commanders(n ** 2 + new_variables_count + 1, len(groups))
        new_variables_count += len(groups)
        binomial_AMO(clauses, commanders)

        for i in range(len(commanders)):
            if len(groups[i]) > 4:
                binary_AMO(clauses, groups[i])
            else:
                binomial_AMO(clauses, groups[i])
            clauses.append(groups[i] + [-commanders[i]])
            for j in range(len(groups[i])):
                clauses.append([commanders[i], -groups[i][j]])

def exactly_one(clauses, variables):
    clauses.append(variables)
    at_most_one(clauses, variables)


def generate_clauses(n, variables):
    clauses = []

    # Exactly one queen in each row
    for row in range(n):
        exactly_one(clauses, variables[row])

    # Exactly one queen in each column
    for col in range(n):
        exactly_one(clauses, [variables[row][col] for row in range(n)])


    # At most one queen in each diagonal
    for i in range(1, n):
        diagonal = []
        row = i
        col = 0
        while row >= 0 and col < n:
            diagonal.append(variables[row][col])
            row -= 1
            col += 1
        at_most_one(clauses, diagonal)

    for j in range(1, n - 1):
        diagonal = []
        row = n - 1
        col = j
        while row >= 0 and col < n:
            diagonal.append(variables[row][col])
            row -= 1
            col += 1
        at_most_one(clauses, diagonal)

    for i in range(n - 1):
        diagonal = []
        row = i
        col = 0
        while row < n and col < n:
            diagonal.append(variables[row][col])
            row += 1
            col += 1
        at_most_one(clauses, diagonal)


    for j in range(1, n - 1):
        diagonal = []
        row = 0
        col = j
        while row < n and col < n:
            diagonal.append(variables[row][col])
            row += 1
            col += 1
        at_most_one(clauses, diagonal)

    return clauses


def solve_n_queens(n):
    variables = generate_variables(n)
    clauses = generate_clauses(n, variables)

    solver = Glucose3()
    for clause in clauses:
        solver.add_clause(clause)

    if solver.solve():
        model = solver.get_model()
        return [[int(model[i * n + j] > 0) for j in range(n)] for i in range(n)]
    else:
        return None


def print_solution(solution):
    if solution is None:
        print("No solution found.")
    else:
        for row in solution:
            print(" ".join("Q" if cell else "." for cell in row))


n = 128
solution = solve_n_queens(n)
print_solution(solution)


. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Q . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Q . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Q . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

Sequential encounter encoding

- Idea:
- Variables: $X_1 X_2, \ldots, X_n$ and new (n-1) variables $\mathrm{S}_1, \mathrm{S}_2, \ldots, \mathrm{S}_{\mathrm{n}-1}$
- $S_i=$ True iff existing a Truth variable in $\left(X_1, X_2, \ldots, X_i\right)$
- AMO Clauses: $X_1+X_2+\ldots+X_n<=1$
    - $X_1 \rightarrow S_1 \wedge$
    - $\left(X_i \vee S_{i-1}\right) \rightarrow S_i \wedge$
- $\left(S_{i-1} \rightarrow \neg X_i\right) \wedge(1<i<n)$
    - $\left(S_{n-1} \rightarrow \neg X_n\right)$
    - Requires: (n-1) new variables, (3n-4) clauses


- For $n=4\left(X_1, X_2, X_3, X_4\right)$
- New variables $S_1, S_2, S_3$
    - AMO Encoding:
    - $x_1 \rightarrow s_1 \wedge$
    - $\left(X_2 \vee S_1\right) \rightarrow S_2 \wedge\left(S_1 \rightarrow \neg X_2\right) \wedge$
- $\left(X_3 \vee S_2\right) \rightarrow S_3 \wedge\left(S_2 \rightarrow \neg X_3\right) \wedge$
    - $\left(S_3 \rightarrow \neg X_4\right)$
    - ALO Encoding: $\left(X_1 \vee X_2 \vee X_3 \vee X_4\right)$


In [ ]:
# sequential

new_variables_count = 0
def generate_variables(n):
    return [[i * n + j + 1 for j in range(n)] for i in range(n)]


def generate_variables(n):
    return [[i * n + j + 1 for j in range(n)] for i in range(n)]

def generate_new_variables(new_variables_count, length):
    start = n ** 2 + new_variables_count
    return [i for i in range(start + 1,start + length)]


def at_most_one(clauses, variables):
    global new_variables_count
    new_variables = generate_new_variables(new_variables_count, len(variables))
    new_variables_count += (len(variables) - 1)
    clauses.append([-variables[0], new_variables[0]])
    for i in range(1, len(variables) - 1):
        clauses.append([-variables[i], new_variables[i]])
        clauses.append([-new_variables[i - 1], new_variables[i]])
        clauses.append([-new_variables[i - 1], -variables[i]])
    clauses.append([-new_variables[len(variables) - 2], -variables[len(variables) - 1]])

def exactly_one(clauses, variables):
    clauses.append(variables)
    at_most_one(clauses, variables)


def generate_clauses(n, variables):
    clauses = []

    # Exactly one queen in each row
    for row in range(n):
        exactly_one(clauses, variables[row])

    # Exactly one queen in each column
    for col in range(n):
        exactly_one(clauses, [variables[row][col] for row in range(n)])


    # At most one queen in each diagonal
    for i in range(1, n):
        diagonal = []
        row = i
        col = 0
        while row >= 0 and col < n:
            diagonal.append(variables[row][col])
            row -= 1
            col += 1
        at_most_one(clauses, diagonal)

    for j in range(1, n - 1):
        diagonal = []
        row = n - 1
        col = j
        while row >= 0 and col < n:
            diagonal.append(variables[row][col])
            row -= 1
            col += 1
        at_most_one(clauses, diagonal)

    for i in range(n - 1):
        diagonal = []
        row = i
        col = 0
        while row < n and col < n:
            diagonal.append(variables[row][col])
            row += 1
            col += 1
        at_most_one(clauses, diagonal)

    for j in range(1, n - 1):
        diagonal = []
        row = 0
        col = j
        while row < n and col < n:
            diagonal.append(variables[row][col])
            row += 1
            col += 1
        at_most_one(clauses, diagonal)

    return clauses


def solve_n_queens(n):
    variables = generate_variables(n)
    clauses = generate_clauses(n, variables)

    solver = Glucose3()
    for clause in clauses:
        solver.add_clause(clause)

    if solver.solve():
        model = solver.get_model()
        return [[int(model[i * n + j] > 0) for j in range(n)] for i in range(n)]
    else:
        return None


def print_solution(solution):
    if solution is None:
        print("No solution found.")
    else:
        for row in solution:
            print(" ".join("Q" if cell else "." for cell in row))


n = 64
solution = solve_n_queens(n)
print_solution(solution)

. . . . Q . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. Q . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . Q . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . Q . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Q
. . . . . . . . . . . . . . . . . . . . . . . . . Q . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Q . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

Product encoding

- Idea: Decompose AMO $\left(X_1, X_2, \ldots, X_n\right)$ into:
  AMO $\left(r_1, r_2, \ldots, r_p\right)$ and AMO $\left(c_1, c_2, \ldots, c_q\right)$
    - A grid with $n$ points $(p \times q): p=[\operatorname{sqrt}(n)], q=[n / p]$
    - One point for each variable $\left(X_1, X_2, \ldots, X_n\right)$
  - $X_1 \Leftrightarrow\left(r_1, c_1\right), X_2 \Leftrightarrow\left(r_1, c_2\right), \ldots, X_n \Leftrightarrow\left(r_p, c_k\right)$

$$
k=n-(p-1)^* q
$$

  - Constraints at most one row and at most one column
- AMO $\left(X_1, X_2, \ldots, X_n\right)$ :
  - AMO $\left(r_1, r_2, \ldots, r_p\right)$ and AMO $\left(c_1, c_2, \ldots, c_q\right)$
  - $X_i \rightarrow r_u \wedge c_v \quad$ where $X_i=\left(r_u, v_u\right)$


- For $n=5\left(X_1, X_2, X_3, X_4, X_5\right)$
    - $p=[\operatorname{sqrt}(5)]=3, v=[5 / 3]=2$
    - $\left(r_1, r_2, r_3\right) \times\left(c_1, c_2\right)$
- $\operatorname{AMO}\left(X_1, X_2, X_3, X_4, X_5\right)$ :
    - $r_1 \rightarrow \neg r_2 \wedge r_1 \rightarrow \neg r_3 \wedge r_2 \rightarrow \neg r_3 \wedge$
    - $c_1 \rightarrow \neg c_2 \wedge$
    - $x_1 \rightarrow r_1 \wedge x_1 \rightarrow c_1 \wedge x_2 \rightarrow r_1 \wedge x_2 \rightarrow c_2 \wedge$
    - $x_3 \rightarrow r_2 \wedge x_3 \rightarrow c_1 \wedge x_4 \rightarrow r_2 \wedge x_4 \rightarrow c_2 \wedge$
    - $x_5 \rightarrow r_3 \wedge x_5 \rightarrow c_1$

In [11]:
# product

new_variables_count = 0

def generate_variables(n):
    return [[i * n + j + 1 for j in range(n)] for i in range(n)]


def binomial_AMO(clauses, variables):
    for i in range(0, len(variables)):
        for j in range(i + 1, len(variables)):
            clauses.append([-variables[i], -variables[j]])
    return clauses

def generate_matrix(start, variables_length):
    p = math.ceil(math.sqrt(variables_length))
    q = math.ceil(n / p)
    return [
        [i for i in range(start, start + p)],
        [j for j in range(start + p, start + p + q)]
    ]


def generate_binary_combinations(n):
    binary_combinations = []
    for i in range(1 << n):
        binary_combinations.append(format(i, '0' + str(n) + 'b'))
    return binary_combinations


def binary_encoding(clauses, target, new_variables):
    for i in range(len(new_variables)):
        clauses.append([-target, new_variables[i]])

def generate_new_variables(end, length):
    return [i for i in range(end + 1, end + math.ceil(math.log(length, 2)) + 1)]

def binary_AMO(clauses, variables):
    global new_variables_count
    temp_new_variables = generate_new_variables(n ** 2 + new_variables_count, len(variables))
    new_variables_count += len(temp_new_variables)
    binary_combinations = generate_binary_combinations(len(temp_new_variables))

    for i in range(len(variables)):
        combination = binary_combinations[i]
        temp_clause = []
        for j in range(len(combination) - 1, -1, -1):
            index = len(combination) - j - 1
            temp_clause.append({True: temp_new_variables[index], False: -temp_new_variables[index]} [int(combination[j]) == 1])

        binary_encoding(clauses, variables[i], temp_clause)

def at_most_one(clauses, variables):
    global new_variables_count
    matrix = generate_matrix(n ** 2 + new_variables_count + 1, len(variables))
    row = matrix[0]
    col = matrix[1]
    new_variables_count += len(row) + len(col)

    if(len(variables) > 10):
        binary_AMO(clauses, row)
        binary_AMO(clauses, col)
    else:
        binomial_AMO(clauses, row)
        binomial_AMO(clauses, col)

    x = 0
    for i in range(len(row)):
        for j in range(len(col)):
            clauses.append([-variables[x], row[i]])
            clauses.append([-variables[x], col[j]])
            x += 1
            if x == len(variables): break
        if x == len(variables): break

def exactly_one(clauses, variables):
    clauses.append(variables)
    at_most_one(clauses, variables)

def generate_clauses(n, variables):
    clauses = []

    # Exactly one queen in each row
    for row in range(n):
        exactly_one(clauses, variables[row])

    # Exactly one queen in each column
    for col in range(n):
        exactly_one(clauses, [variables[row][col] for row in range(n)])

    # At most one queen in each diagonal
    for i in range(1, n):
        diagonal = []
        row = i
        col = 0
        while row >= 0 and col < n:
            diagonal.append(variables[row][col])
            row -= 1
            col += 1
        at_most_one(clauses, diagonal)

    for j in range(1, n - 1):
        diagonal = []
        row = n - 1
        col = j
        while row >= 0 and col < n:
            diagonal.append(variables[row][col])
            row -= 1
            col += 1
        at_most_one(clauses, diagonal)

    for i in range(n - 1):
        diagonal = []
        row = i
        col = 0
        while row < n and col < n:
            diagonal.append(variables[row][col])
            row += 1
            col += 1
        at_most_one(clauses, diagonal)

    for j in range(1, n - 1):
        diagonal = []
        row = 0
        col = j
        while row < n and col < n:
            diagonal.append(variables[row][col])
            row += 1
            col += 1
        at_most_one(clauses, diagonal)

    return clauses


def solve_n_queens(n):
    variables = generate_variables(n)
    clauses = generate_clauses(n, variables)

    solver = Glucose3()
    for clause in clauses:
        solver.add_clause(clause)

    if solver.solve():
        model = solver.get_model()
        return [[int(model[i * n + j] > 0) for j in range(n)] for i in range(n)]
    else:
        return None


def print_solution(solution):
    if solution is None:
        print("No solution found.")
    else:
        for row in solution:
            print(" ".join("Q" if cell else "." for cell in row))


n = 128
solution = solve_n_queens(n)
print_solution(solution)


. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Q . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Q . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Q . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Q . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 